In [9]:
import requests
import base64
import pandas as pd
import numpy as np
import json

In [16]:
with open('config.json', 'r') as f:
    config = json.load(f)

{'CLIENT_ID': 'a7c0ec514e3d4382acfa62c3417957ee',
 'CLIENT_SECRET': 'cb8a259b23d24d1aacd1d1b5b531bd08'}

In [13]:
CLIENT_ID = config['CLIENT_ID']
CLIENT_SECRET = config

# POST

In [14]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

CLIENT_CREDENTIALS = f'{CLIENT_ID}:{CLIENT_SECRET}'
CLIENT_CREDENTIALS_B64 = base64.b64encode(CLIENT_CREDENTIALS.encode())

TOKEN_DATA = {
    "grant_type": "client_credentials"
}

TOKEN_HEADERS = {
    "Authorization": f"Basic {CLIENT_CREDENTIALS_B64.decode()}"
}

In [20]:
REQUEST = requests.post(AUTH_URL, data = TOKEN_DATA, headers = TOKEN_HEADERS)
response_data = REQUEST.json()
access_token = response_data['access_token']
expires_in = response_data['expires_in']

In [18]:
access_token

'BQCFreDAIYadiOjfmbjG3nbJL89z_EPJQ2fS6YrvzljeNAmuhT8QLBEjpT1W66l52MTpa8CyIVJgD2frN7hvNqN7RPtF5jbxJ1UGcVjLUVmwlgFBTf0'

In [21]:
expires_in

3600

# Spotipy

In [78]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [79]:
# Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [80]:
# We get all tracks in a playlist
playlist_link = "https://open.spotify.com/playlist/0JNQmyPN5iA9e2iYNwE1wy?si=61bc088cae494503"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [81]:
# We can loop over all tracks and get information about them
keys = ["artist_name", "artist_pop", "track_name", "track_pop", "album", "key", "mode", "tempo", "duration_ms", "time_signature", "energy", "danceability", "instrumentalness"]
artist_keys = ["artist_name", "artist_pop", "track_name", "track_pop", "album"]
audio_keys = ["key", "mode", "tempo", "duration_ms", "time_signature", "energy", "danceability", "instrumentalness"]
dict_data = {key: np.array([]) for key in keys}
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    
    # We get audio features
    audio_features = sp.audio_features(track_uri)[0]
    
    # We save the results
    dict_data["artist_name"] = np.append(dict_data["artist_name"], artist_name)
    dict_data["artist_pop"] = np.append(dict_data["artist_pop"], artist_pop)
    dict_data["track_name"] = np.append(dict_data["track_name"], track_name)
    dict_data["track_pop"] = np.append(dict_data["track_pop"], track_pop)
    dict_data["album"] = np.append(dict_data["album"], album)
    
    for audio_key in audio_keys:
        dict_data[audio_key] = np.append(dict_data[audio_key], audio_features[audio_key])
    
    # Prints
    print(artist_name, "-", track_name)

AVAION - Pieces
SIDEPIECE - Together
Kaskade - Turn It Down v3
JLV - Chemical Days
Swedish House Mafia - Redlight
John Summit - Deep End - SIDEPIECE Remix
ARTBAT - Horizon
Michael Calfan - No Lie
Jess Ball - Messed Up
SOMMA - Body on Fire
Nazzereene - Where I Belong
Don Diablo - Bad
Lost Frequencies - Where Are You Now
Tinlicker - Because You Move Me
Shouse - Love Tonight - David Guetta Remix Edit
Sofi Tukker - Sun Came Up
Hayden James - On Your Own (feat. Elderbrook)
ACRAZE - Do It To It
The Chainsmokers - High - Don Diablo Remix
Ryan Shepherd - Never Give Up On You
Dom Dolla - Strangers (with Mansionair)
MIME - DO IT AGAIN
Icona Pop - Off Of My Mind
Martin Garrix - Pressure (feat. Tove Lo)
EDX - Take Me Home (feat. Jess Ball)
The Weeknd - Sacrifice
Tim Morrison - Revolver
Flume - Say Nothing (feat. MAY-A)
MEDUZA - Tell It To My Heart
Swedish House Mafia - Moth To A Flame (with The Weeknd)
Dylan Jett - Red
Elderbrook - Inner Light
Nazzereene - That's How I Like It
John Summit - Human 

In [82]:
df = pd.DataFrame(dict_data)
df.set_index("artist_name", inplace = True)

In [84]:
df.to_csv("hypno_deep.csv", sep = ";")